In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# ==============================================================================
#  1. Installation
# ==============================================================================
# This cell installs all the required Python libraries for this notebook.

# - google-cloud-aiplatform: The client library for Vertex AI and Agent Engine.
# - google-adk: The Agent Development Kit for building the agent.
# - pandas: Used for displaying database query results in a clean table format.
# - pg8000: A Python driver needed to connect to the PostgreSQL database.

# Note: We remove python-dotenv as it's no longer needed.
%pip install -q google-cloud-aiplatform[reasoning_engines] google-adk pandas pg8000

print("✅ All required libraries are installed.")

In [ ]:
import os
import vertexai
import sqlite3
import pandas as pd
from google.adk.agents import Agent
from google.adk.sessions import BaseSessionService, DatabaseSessionService
from vertexai.preview.reasoning_engines import AdkApp
from vertexai import agent_engines

In [ ]:
# ==============================================================================
#  Set Your Google Cloud Configuration
# ==============================================================================
# This cell sets the core configuration for your Google Cloud environment.

# ACTION REQUIRED:
# Replace the placeholder values below with your actual Project ID, Location,
# and the name of a Cloud Storage bucket for staging.

import os

os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
os.environ["STAGING_BUCKET"] = "gs://your-staging-bucket"

print("✅ Google Cloud environment variables configured.")

In [ ]:
# Loading the first env variables

PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]

In [ ]:
# Initialize the Vertex AI SDK.
# This authenticates your environment and sets the default project, location,
# and staging bucket for all subsequent Vertex AI calls in this session.

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [ ]:
# ==============================================================================
#  Define the Core Agent
# ==============================================================================
# The Agent is the brain of our application. It combines a specific Gemini model
# with a detailed instruction prompt that defines its personality, goals, and
# constraints.

# First, define the detailed instructions for the agent's behavior.
simple_prompt = """You are a helpful example bot that tries to answer anything the client wants.
Help the client by answering his questions. Be respectful and kind, but if the client asks something bad feel free to deny it"""

# Then, create the Agent instance, passing the prompt and other metadata.
root_agent = Agent(
    name="rag_agent",
    model="gemini-2.0-flash",
    description=(
        "Help the user by answering questions."
    ),
    instruction=simple_prompt,
)

## Option 1: Using SQLite (Local Development & Testing)

To get started, we'll use the simplest method for storing conversation history: a local **SQLite** database.

### Why Use SQLite?
* **Zero Setup**: It doesn't require any separate database server, authentication, or network configuration. The ADK handles everything for you.
* **File-Based**: Your entire database is a single file (`sessions.db`) that gets created in your project directory, making it easy to inspect or delete.
* **Perfect for Development**: It's the ideal choice for quickly building and testing your agent on your local machine.

In the next cell, we will define a builder function that tells our agent to use this SQLite setup for its memory.

In [ ]:
# ==============================================================================
#  Define a Session Service Builder (Option 1: SQLite)
# ==============================================================================
# To remember conversations, the agent needs a session service. To start with,
# we will use the simplest option: a service that stores conversation history
# in a local SQLite database file (`sessions.db`).

# This setup is perfect for getting started quickly, as it requires no
# external database configuration. Later, we can easily swap this function with
# one that connects to a more robust cloud database.

# This function acts as a "builder" that the AdkApp will call to get the
# session manager it needs for handling conversation memory.


def create_database_session_service_sqlite() -> BaseSessionService:
    """
    Creates and configures the DatabaseSessionService for SQLite.
    This function acts as a "builder" that the AdkApp can call whenever
    it needs to create a session service instance.
    """
    db_connection_string = "sqlite:///sessions.db"

    print(f"Session service configured to use URL: {db_connection_string}")
    return DatabaseSessionService(db_url=db_connection_string)


In [ ]:
# ==============================================================================
#  Create the Main Application
# ==============================================================================
# The AdkApp serves as the main entry point for interacting with our agent.
# We initialize it by bringing together our agent's logic and the session
# management needed to store conversation history.

# We pass it two key components:
#  1. agent: The `root_agent` we defined earlier, which contains the prompt and model.
#  2. session_service_builder: The function that tells the app how to create
#     our SQLite session service for managing conversation memory.

app = AdkApp(
    agent=root_agent,
    session_service_builder=create_database_session_service_sqlite,
)

In [ ]:
# ==============================================================================
#  Create a New Conversation Session
# ==============================================================================
# Before we can start chatting, we need to create a "session," which represents
# a single, stateful conversation that the agent can remember.

# First, we define an ID for the user who is interacting with the agent.
user_id = "local_user_demo"

# Then, we call `app.create_session()` to start the conversation. This creates
# a new record in our database and returns a session object.
session = app.create_session(user_id=user_id)

# We print the unique `session.id`. We must use this ID in all subsequent
# messages to continue this specific conversation.
print(f"Created a new session for user '{user_id}' with session ID: {session.id}")

In [ ]:
# ==============================================================================
#  Send a Message and Stream the Response
# ==============================================================================
# This is the final step where we send our message to the agent and get its
# response back.

print("\n--- User's Message ---")
message = "Hello whats up?"
print(f"You: {message}\n")

print("--- Agent's Response ---")
for event in app.stream_query(user_id = user_id, 
                              session_id=session.id, 
                              message=message):
    print(event['content'])

In [ ]:
# ==============================================================================
#  A Look Inside the SQLite Database
# ==============================================================================
# Now that our agent has had a conversation, the DatabaseSessionService has saved the entire history into our sessions.db file.
# We have the following tables: app_states, events, sessions, user_states
# Lets take a look inside each one of them


In [ ]:
# First, we set up our database connection. This cell points to the
# sessions.db file and creates a live connection to it that we'll use
# in the following cells to run queries.

# Define the path to your SQLite database file.
DB_FILE = "sessions.db"

# Set pandas to display all column content without truncating
pd.set_option('display.max_colwidth', None)

# Connect to the SQLite database
conn = sqlite3.connect(DB_FILE)

In [ ]:
# Now, let's query and display the contents of the 'sessions' table.
# This table stores the main record for each conversation, including the
# full history of turns between the user and the agent.

print("--- Contents of the 'sessions' table ---")
sessions_df = pd.read_sql_query("SELECT * FROM sessions", conn)
display(sessions_df)

In [ ]:
# Next, we'll look at the 'events' table. The ADK uses this table
# to log specific actions or events that occur while the agent is
# processing a request.

print("--- Contents of the 'events' table ---")
events_df = pd.read_sql_query("SELECT * FROM events", conn)
display(events_df)

In [ ]:
# Next, let's look at the `app_states` table. The ADK uses this to store
# the agent's internal "application state" or tool memory that needs to
# persist across turns but isn't part of the chat history itself.

print("--- Contents of the 'app_states' table ---")
app_states_df = pd.read_sql_query("SELECT * FROM app_states", conn)
display(app_states_df)

In [ ]:
# Finally, we'll inspect the `user_states` table. This table holds
# long-term memory about a specific user, like their preferences, which
# can be recalled across many different conversations or sessions.

print("--- Contents of the 'user_states' table ---")
user_states_df = pd.read_sql_query("SELECT * FROM user_states", conn)
display(user_states_df)

## Option 2: Using a Cloud SQL Database (Production Setup)

While SQLite is great for getting started, a managed cloud database like **Cloud SQL** is the right choice for a more robust or production-ready application.

### Why Use Cloud SQL?
* **Scalability**: It can handle many more simultaneous connections than a local SQLite file.
* **Persistence & Reliability**: As a managed service, Google handles backups, failovers, and maintenance, making your data much safer.
* **Accessibility**: Your deployed applications (e.g., on Agent Engine) can connect to it from anywhere within your Google Cloud project.

The following steps will guide you through creating a PostgreSQL instance on Cloud SQL using `gcloud` commands.

### Step 1: Enable APIs

Before creating an instance, you need to ensure the necessary APIs are enabled for your project. The following command enables the Cloud SQL Admin API (for managing instances) and the Vertex AI API.

In [ ]:
# Run this cell to enable the required Google Cloud services.
!gcloud services enable sqladmin.googleapis.com aiplatform.googleapis.com --project=$PROJECT_ID

print("✅ APIs enabled.")

### Step 2: Create the Cloud SQL Instance

Next, we'll create the Cloud SQL instance. This command will provision a new PostgreSQL database.

* **Instance ID**: A unique name for your instance, like `adk-agent-db`.
* **Password**: You will be prompted to set a strong password for the default `postgres` user. **Important: Save this password somewhere secure!** You will need it for your `.env` file.
* **Region**: To get the best performance, choose the **same region** where your other Vertex AI resources are located (e.g., `us-central1`).
* **Database Version**: We'll use PostgreSQL 13, but you can choose another supported version.
* **Tier**: The `db-g1-small` is a cost-effective option for development and testing.

Replace `adk-agent-db` with your desired instance ID in the command below.

In [ ]:
import getpass
import shlex
import os

# Set your desired instance ID and region.
# Make sure the region matches your project's location.
INSTANCE_ID = "my-instance-name"
REGION = LOCATION # Using the LOCATION variable from earlier

# This command can take a few minutes to complete.
print(f"Creating Cloud SQL instance '{INSTANCE_ID}'. This will take several minutes...")
!gcloud sql instances create {INSTANCE_ID} \
    --database-version=POSTGRES_13 \
    --tier=db-g1-small \
    --region={REGION} \
    --project={PROJECT_ID}
print("Instance created.")

# --- Set Password Interactively ---
# Use getpass to create a secure password prompt in the notebook output.
db_password = getpass.getpass("Enter a secure password for the 'postgres' user: ")

# Set the environment variable immediately for later use.
os.environ["DB_PASSWORD"] = db_password
print("DB_PASSWORD environment variable has been set in this session.")

# Securely quote the password for the shell command
quoted_password = shlex.quote(db_password)

# Set the password using the --password flag.
!gcloud sql users set-password postgres \
    --instance={INSTANCE_ID} \
    --project={PROJECT_ID} \
    --password={quoted_password}

print(f"✅ Cloud SQL instance '{INSTANCE_ID}' created and password set for 'postgres' user.")

### Step 3: Create a Database

Once your instance is running, you need to create a specific database inside it for your agent's sessions. We'll name it `agent_sessions`.

In [ ]:
DATABASE_NAME = "my-database-name"

!gcloud sql databases create {DATABASE_NAME} \
    --instance={INSTANCE_ID} \
    --project={PROJECT_ID}

print(f"✅ Database '{DATABASE_NAME}' created in instance '{INSTANCE_ID}'.")

### Step 4: Get the Instance Connection Name

The instance connection name is a unique identifier for your database instance that is required for connecting securely, especially with the Cloud SQL Auth Proxy. The following command retrieves this name.

In [ ]:
# Get the connection name and save it to a variable to use later.
CONNECTION_NAME_VAR = !gcloud sql instances describe {INSTANCE_ID} --project={PROJECT_ID} --format='value(connectionName)'
CONNECTION_NAME = CONNECTION_NAME_VAR[0]

# You will need to add this to your .env file.
print(f"Instance Connection Name: {CONNECTION_NAME}")
print("ACTION: Copy this value and paste it into the 'Set and Load Database Connection Details' cell below.")

### Step 5: Download and Run the Cloud SQL Auth Proxy

To connect your local machine to your Cloud SQL instance securely, you must use the **Cloud SQL Auth Proxy**. This tool creates a secure local connection without needing to configure IP allowlisting.

The next cell automates the process of downloading the correct proxy for your operating system and starting it as a background process. The proxy must be running whenever you want to connect to your database from this notebook.

**Note**: The proxy will run in the background. To stop it, you will need to interrupt or restart the notebook's kernel.

In [ ]:
import os
import platform
import subprocess
import time

# Check if CONNECTION_NAME is defined
if 'CONNECTION_NAME' not in locals():
    raise NameError("CONNECTION_NAME is not defined. Please run the previous cell to get the instance connection name.")

# Determine the correct proxy binary URL
system = platform.system()
machine = platform.machine()
proxy_url = ""
executable_name = "cloud-sql-proxy"

if system == "Linux" and machine == "x86_64":
    proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.linux.amd64"
elif system == "Darwin":
    if machine == "arm64":
        proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.darwin.arm64"
    elif machine == "x86_64":
        proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.darwin.amd64"
elif system == "Windows":
     proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.x64.exe"
     executable_name += ".exe"

if not proxy_url:
    print(f"Unsupported OS/architecture: {system}/{machine}. Please install the Cloud SQL Auth Proxy manually.")
else:
    # Download the proxy
    print(f"Downloading Cloud SQL Auth Proxy for {system}/{machine}...")
    !curl -o {executable_name} {proxy_url}
    
    # Make it executable (if not on Windows)
    if system != "Windows":
        print("Making it executable...")
        !chmod +x {executable_name}

    # Start the proxy in the background
    print(f"Starting proxy for instance: {CONNECTION_NAME}")
    # Using Popen to run it as a non-blocking background process
    proxy_process = subprocess.Popen([f"./{executable_name}", CONNECTION_NAME])
    
    # Give it a moment to start up
    time.sleep(5)
    
    print(f"✅ Cloud SQL Auth Proxy started in the background with PID: {proxy_process.pid}")
    print("You can now proceed to the next cells to connect to the database.")

In [ ]:
# ==============================================================================
#  Set and Load Database Connection Details
# ==============================================================================
# This cell configures and loads the connection details for our Cloud SQL database.
# The DB_PASSWORD was set interactively in the cell that created the instance.

# ACTION REQUIRED:
# 1. Paste the 'Instance Connection Name' you copied from the output of Step 4.

# --- Set Database environment variables ---
os.environ["DB_USER"] = "postgres"
os.environ["DB_NAME"] = "db-name"
os.environ["CONNECTION_NAME"] = f"connection-name" # <-- PASTE CONNECTION NAME HERE
os.environ["DB_HOST"] = "db_host_ip" # example: 127.0.0.1
os.environ["DB_PORT"] = "port_number" # example: 5432

# --- Load variables into the script ---
# We retrieve DB_PASSWORD from the environment, assuming it was set by the getpass prompt earlier.
DB_USER = os.environ["DB_USER"]
DB_PASSWORD = os.environ["DB_PASSWORD"]
DB_NAME = os.environ["DB_NAME"]
CONNECTION_NAME = os.environ["CONNECTION_NAME"]
DB_HOST = os.environ["DB_HOST"]
DB_PORT = os.environ["DB_PORT"]

if not DB_PASSWORD:
    print("⚠️ DB_PASSWORD is not set. Please re-run the 'Create the Cloud SQL Instance' cell.")
else:
    print("✅ Cloud SQL environment variables configured.")

In [ ]:
# ==============================================================================
#  Define a Session Service Builder (Option 2: Cloud SQL)
# ==============================================================================
# This is our second session service builder, designed to connect to the
# Cloud SQL for PostgreSQL instance we configured earlier.

# It takes the database credentials loaded from the .env file and builds a
# standard PostgreSQL connection string. This string points to the Cloud SQL
# Auth Proxy, which must be running locally to securely tunnel the connection.

# We can easily switch the agent's memory from local SQLite to this cloud
# database simply by changing which builder function we use in the next step.

def create_database_session_service_sql() -> BaseSessionService:
    db_url = f"postgresql+pg8000://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

    # You can also pass other SQLAlchemy engine arguments if needed.
    session_db_kwargs = {
        "pool_recycle": 3600,
        "echo": False  # Set to True for debugging, False in production
    }

    print(f"Creating DatabaseSessionService with Cloud SQL URL.")
    return DatabaseSessionService(db_url=db_url, **session_db_kwargs)

In [ ]:
# ==============================================================================
#  Create the App Instance (Cloud SQL Option)
# ==============================================================================
# Now we create a new AdkApp instance, but this time we connect it to our
# Cloud SQL database.

# By simply passing our new `create_database_session_service_sql` builder,
# we swap the agent's memory backend. All conversations handled by this
# `sql_app` object will now be stored in our PostgreSQL database.

sql_app = AdkApp(
    agent=root_agent,
    session_service_builder=create_database_session_service_sql,
)

In [ ]:
# ==============================================================================
#  Create a session for a sample user
# ==============================================================================


user_id = "sql_user_demo"

session = sql_app.create_session(user_id=user_id)

In [ ]:
# ==============================================================================
#  Interact with the Agent (Using Cloud SQL)
# ==============================================================================
# Finally, let's test our new Cloud SQL-backed application.

# We'll use the `sql_app` instance and the new session ID we just created to
# send a message to the agent. Because we are using this app, this new
# conversation turn will be saved to our PostgreSQL database in Cloud SQL.

# After running this, you can use `psql` or the Cloud SQL Studio to query your
# database and see this new conversation appear in the 'sessions' table.

print("\n--- User's Message ---")
message = "Hello whats up?"
print(f"You: {message}\n")


print("--- Agent's Response ---")
for event in sql_app.stream_query(user_id=user_id, session_id=session.id, message=message):
    print(event['content'])